In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/egyptian-real-estate-listings/egypt_real_estate_listings.csv


In [19]:
df = pd.read_csv("/kaggle/input/egyptian-real-estate-listings/egypt_real_estate_listings.csv")

In [21]:
import re, pandas as pd
from sqlalchemy import create_engine  # only if you want SQLite export

# --- load & drop ---
cols_to_drop = ["url", "payment_method", "down_payment"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns]).copy()

df = df.dropna(subset=["price", "size", "bedrooms", "bathrooms", "available_from"])

# --- helpers ---
def to_number_price(x):
    if pd.isna(x): return pd.NA
    s = re.sub(r"[^\d.]", "", str(x))
    return float(s) if s else pd.NA

def to_sqm(s):
    if pd.isna(s): return pd.NA
    s = str(s).lower().strip()
    if "/" in s: s = s.split("/")[-1].strip()
    m = re.search(r"(\d+(?:\.\d+)?)\s*sqm", s)
    if m: return float(m.group(1))
    m = re.search(r"(\d+(?:\.\d+)?)\s*sqft", s)
    if m: return float(m.group(1)) * 0.092903
    m = re.search(r"(\d+(?:\.\d+)?)", s)
    return float(m.group(1)) if m else pd.NA

def normalize_bedrooms(s):
    if pd.isna(s): return pd.NA
    s = str(s).lower()
    nums = [int(n) for n in re.findall(r"\d+", s)]
    total = sum(nums) if nums else 0
    if "maid" in s: total += 1
    return total if total else pd.NA

def normalize_bathrooms(s):
    if pd.isna(s): return pd.NA
    m = re.search(r"(\d+(?:\.\d+)?)", str(s))
    return float(m.group(1)) if m else pd.NA

# --- transform core fields ---
df["price"] = df["price"].apply(to_number_price)
df["size"] = df["size"].apply(to_sqm)
df["bedrooms"] = df["bedrooms"].apply(normalize_bedrooms)
df["bathrooms"] = df["bathrooms"].apply(normalize_bathrooms)

# --- DATE: normalize for SQL QA ---
# 1) parse with day-first (e.g., '31 Aug 2025')
dt = pd.to_datetime(df.get("available_from"), errors="coerce", dayfirst=True)

# 2) store as ISO text 'YYYY-MM-DD' (or NaN -> will become NULL in SQL)
df["available_from"] = dt.dt.strftime("%Y-%m-%d")

# (optional) handy derived fields for Text-to-SQL:
df["available_year"]  = dt.dt.year
df["available_month"] = dt.dt.month
df["available_day"]   = dt.dt.day

# --- enforce dtypes that are nice for SQL ---
df["price"] = df["price"].astype(int)
df["size"] = df["size"].astype(float)
df["bedrooms"] = df["bedrooms"].astype(int)
df["bathrooms"] = df["bathrooms"].astype(int)

/tmp/ipykernel_36/2039977704.py:48: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt = pd.to_datetime(df.get("available_from"), errors="coerce", dayfirst=True)


In [22]:
df = df.head(100)

In [24]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time

# Initialize the client
client = OpenAI(
    api_key="API_KEY",
    base_url="https://api.opentyphoon.ai/v1"
)

def generate_caption(row):
    """
    Call LLM to generate Thai caption/name for one listing row.
    """
    prompt = (
        f"สร้างคำอธิบายสั้นๆ สำหรับประกาศอสังหาริมทรัพย์:\n"
        f"- ราคา: {row['price']}\n"
        f"- ทำเล: {row['location']}\n"
        f"- ประเภท: {row['type']}\n"
        f"- ขนาด: {row['size']}\n"
        f"- ห้องนอน: {row['bedrooms']}\n"
        f"- ห้องน้ำ: {row['bathrooms']}\n"
        f"- เข้าอยู่ได้ตั้งแต่: {row['available_from']} "
        f"({row['available_year']}-{row['available_month']:02d}-{row['available_day']:02d})\n\n"
        f"กรุณาตอบเป็นชื่อ/คำบรรยายที่เหมาะสมเป็นภาษาไทยสั้นๆ"
    )

    try:
        response = client.chat.completions.create(
            model="typhoon-v2.1-12b-instruct",
            messages=[
                {"role": "system", "content": "คุณเป็นผู้ช่วยที่ช่วยเขียนชื่อประกาศอสังหาริมทรัพย์ให้น่าสนใจ ตอบเป็นภาษาไทยสั้นๆ เท่านั้น"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=64,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print("Error:", e)
        return None

# Use tqdm for progress
captions = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating captions"):
    captions.append(generate_caption(row))
    time.sleep(0.5)  # adjust if you hit rate-limits

df["caption"] = captions

Generating captions: 100%|██████████| 100/100 [10:59<00:00,  6.59s/it]


In [26]:
import re
import pandas as pd
from typing import Optional

FIRST_BULLET_PATTERN = re.compile(
    r"""
    (?:^\s*\*\s+|^\s*-\s+|^\s*\d+\.\s+|^\s*•\s+)  # bullet markers: *, -, 1., •
    (.+?)                                         # the text we want
    \s*$                                          # trim right
    """,
    re.MULTILINE | re.VERBOSE
)

def pick_first_option(text: Optional[str]) -> str:
    """
    From an LLM block like:
        **ตัวเลือกชื่อ:**
        *   ชาเลต์หรู สวนทะเลทรายกูนา
        *   พักผ่อนริมทะเล ชาเลต์สวย กูน่า
        *   ชาเลต์ 2 นอน วิวทะเลแดงสุดปัง!

        **คำบรรยายสั้นๆ:**
    return -> 'ชาเลต์หรู สวนทะเลทรายกูนา'

    If no bullet exists, return the first non-empty line after stripping markdown.
    """
    if not isinstance(text, str) or not text.strip():
        return ""

    s = text.strip()

    # 1) If there's a “ตัวเลือกชื่อ” section, narrow to the lines after it
    m = re.search(r"ตัวเลือกชื่อ\s*:?\s*\**", s)
    if m:
        s = s[m.end():].strip()

    # 2) Find first bullet line
    m = FIRST_BULLET_PATTERN.search(s)
    if m:
        return m.group(1).strip()

    # 3) Fallbacks: strip common markdown and take first non-empty line
    s = re.sub(r"\*\*(.*?)\*\*", r"\1", s)         # remove bold
    s = re.sub(r"`{1,3}.*?`{1,3}", "", s, flags=re.DOTALL)  # remove code spans/blocks
    for line in s.splitlines():
        line = line.strip()
        if line and not line.startswith(("*", "-", "•")) and not re.match(r"^\d+\.\s+", line):
            return line
    return ""

from tqdm import tqdm
tqdm.pandas(desc="Cleaning captions")
df["caption"] = df["caption"].progress_apply(pick_first_option)

Cleaning captions: 100%|██████████| 100/100 [00:00<00:00, 66639.72it/s]


In [27]:
df.head(100).to_csv("cleaned_data100.csv")